In [ ]:
### Step 1: Read inflow data

# Read data
import pandas as pd
pd. set_option('display.max_rows', None) 
inflow_data = "data/flow_data.csv"
inflow_data = pd.read_csv(inflow_data, parse_dates=True, index_col=0)

# Preprocessing
inflow_data = inflow_data.fillna(method='ffill')
inflow_data = inflow_data.fillna(method='bfill')
inflow_data.info()

# Add some noise
import numpy as np
noise = np.random.normal(0, 0.01, inflow_data.shape) 
inflow_data = inflow_data + noise

# Normalize to 0-1 range
inflow_data=(inflow_data-inflow_data.min())/(inflow_data.max()-inflow_data.min())

In [ ]:
### Step 1a: Make data stationary
from statsmodels.tsa.stattools import adfuller

def adfuller_test(series, signif=0.05):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    if p_value <= signif:
        return True
    else:
        return False

# Difference until data is stationary
stationary = False
while not stationary:
    stationary = True
    for name, column in inflow_data.iteritems():
        if not adfuller_test(column, 0.05):
            print("data not stationary, differencing...")
            stationary = False
            inflow_data = inflow_data.diff().dropna()
            break

### TODO Step 1b: Split into training and testing data


In [ ]:
# Step 2: Create Vector Autoregressive Model and determine best lag order
from statsmodels.tsa.vector_ar.var_model import VAR
model = VAR(inflow_data, freq='H')
x = model.select_order(maxlags=32)
x.summary()

In [ ]:
# Step 3: Train model
trained_model = model.fit(1) # Put desired lag order here

In [ ]:
# Step 4: Evaluate model
print("Params shape: "+ str(trained_model.params.shape))
print("Inflow data shape: " + str(inflow_data.shape))
print("Final prediction error: ", trained_model.fpe)

In [ ]:
# TODO Step 4b: Do Forecast with trained model